In [2]:
import base64
import json
import random
import pandas as pd
import requests
import tensorflow as tf

In [3]:
df =  pd.read_csv(r"D:\tugas\mlops\coba_pylint\data\healthcare-dataset-stroke-data.csv")
df

,id,gender,hypertension,heart_disease,ever_married,work_type,Residence_type,smoking_status,age,avg_glucose_level,bmi,stroke
0,9046,Male,no,yes,Yes,Private,Urban,formerly smoked,67,228.69,36.6,1
1,51676,Female,no,no,Yes,Self-employed,Rural,never smoked,61,202.21,28.9,1
2,31112,Male,no,yes,Yes,Private,Rural,never smoked,80,105.92,32.5,1
3,60182,Female,no,no,Yes,Private,Urban,smokes,49,171.23,34.4,1
4,1665,Female,yes,no,Yes,Self-employed,Rural,never smoked,79,174.12,24.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,yes,no,Yes,Private,Urban,never smoked,80,83.75,28.9,0
5106,44873,Female,no,no,Yes,Self-employed,Urban,never smoked,81,125.20,40.0,0
5107,19723,Female,no,no,Yes,Self-employed,Rural,never smoked,35,82.99,30.6,0
5108,37544,Male,no,no,Yes,Private,Rural,formerly smoked,51,166.29,25.6,0


In [4]:
print(df.dtypes)

id                     int64
gender                object
hypertension          object
heart_disease         object
ever_married          object
work_type             object
Residence_type        object
smoking_status        object
age                    int64
avg_glucose_level    float64
bmi                  float64
stroke                 int64
dtype: object


In [5]:
# Cell 2: Memilih data acak untuk prediksi
columns = df.columns.values
rand = random.randint(0, len(df) - 1)

# Mengambil fitur
features = df.values[rand]

# Membuat dictionary input
inputs = {key: value for key, value in zip(columns, features)}
inputs

{'id': 8655,
 'gender': 'Female',
 'hypertension': 'no',
 'heart_disease': 'yes',
 'ever_married': 'Yes',
 'work_type': 'Self-employed',
 'Residence_type': 'Urban',
 'smoking_status': 'never smoked',
 'age': 51,
 'avg_glucose_level': 100.96,
 'bmi': 33.4,
 'stroke': 0}

In [6]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[bytes(value, "utf-8")]),
    )

def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=[value]),
    )

def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(value=[value]),
    )

In [7]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, str):
            feature_spec[keys] = string_feature(values)
        elif isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        else:
            feature_spec[keys] = int_feature(values)

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    return json.dumps(
        {
            "signature_name": "serving_default",
            "instances": result,
        }
    )

In [10]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)

    endpoint = "https://stroke-pred-deploy-production.up.railway.app/v1/models/stroke-model:predict"
    response = requests.post(endpoint, data=json_data, headers={"Content-Type": "application/json"})
    prediction = response.json()["predictions"][0][0]

    if prediction > 0.7:
        return "Stroke"
    else:
        return "Safe"

prediksi = make_predictions(inputs)
print(f'Hasil Prediksi: {prediksi}')
print('Fitur Input:', inputs)

Hasil Prediksi: Safe
Fitur Input: {'id': 8655, 'gender': 'Female', 'hypertension': 'no', 'heart_disease': 'yes', 'ever_married': 'Yes', 'work_type': 'Self-employed', 'Residence_type': 'Urban', 'smoking_status': 'never smoked', 'age': 51, 'avg_glucose_level': 100.96, 'bmi': 33.4, 'stroke': 0}
